In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [87]:
# эмоции
emotions = ['anger', 'disgust',	'fear',	'joy',	'sadness',	'surprise',	'neutral']

In [21]:
# Загрузка данных из Excel файла
input_file = 'C:/Users/vasiliy.vorobev/Documents/Разное/Обучение/Проект/Лог регрессия/emotions_data.xlsx' # Замените на путь к вашему файлу
df = pd.read_excel(input_file, usecols=['text', 'anger', 'disgust',	'fear',	'joy',	'sadness',	'surprise',	'neutral'])
df

,text,anger,disgust,fear,joy,sadness,surprise,neutral
0,"Моя любимая еда — это все, что мне не приходил...",0,0,0,0,0,0,1
1,"Теперь, если он покончит с собой, все будут ду...",0,0,0,0,0,0,1
2,КАКОГО НАХРАНА БЭЙЛЕССКАЯ ИЗОИНЦИЯ?,1,0,0,0,0,0,0
3,Чтобы она почувствовала угрозу,0,0,1,0,0,0,0
4,Грязные южные дрочники,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
43405,"Добавил, приятель, у меня только что появился ...",0,0,0,1,0,0,0
43406,"Всегда думал, что это смешно, но является ли э...",0,0,0,0,0,1,0
43407,"О чем ты говоришь? Во всем плохом, что произош...",1,0,0,0,0,0,0
43408,"Больше похоже на крещение, с сексуальными резу...",0,0,0,1,0,0,0


In [41]:
# статистика
df.info()

#df.describe()
#emotion_counts = df[emotion].value_counts()
#print("Статистика ")
#print('Статистика - ', emotion_counts)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43410 entries, 0 to 43409
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      43407 non-null  object
 1   anger     43410 non-null  int64 
 2   disgust   43410 non-null  int64 
 3   fear      43410 non-null  int64 
 4   joy       43410 non-null  int64 
 5   sadness   43410 non-null  int64 
 6   surprise  43410 non-null  int64 
 7   neutral   43410 non-null  int64 
dtypes: int64(7), object(1)
memory usage: 2.6+ MB


,text,anger,disgust,fear,joy,sadness,surprise,neutral
8116,NaN,0,0,0,1,0,0,0
13100,NaN,0,0,0,0,0,0,1
14719,NaN,0,0,0,0,0,0,1


In [45]:
# Удаление Nan
df = df.dropna()
df[df['text'].isnull()]

,text,anger,disgust,fear,joy,sadness,surprise,neutral


In [75]:
# 3. Провести предобработку поля text для дальнейшего обучения ML модели
# Убедитесь, что вы скачали необходимые наборы данных для NLTK:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Привести к нижнему регистру
    text = text.lower()
    # Удалить лишние символы
    text = re.sub("[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+",' ', text)
    # Удалить стоп слова и лемматизировать текст
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['processed_text'] = df['text'].apply(preprocess_text)
df

<>:14: SyntaxWarning: invalid escape sequence '\]'
<>:14: SyntaxWarning: invalid escape sequence '\]'
C:\Users\vasiliy.vorobev\AppData\Local\Temp\ipykernel_6096\3470394433.py:14: SyntaxWarning: invalid escape sequence '\]'
  text = re.sub("[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+",' ', text)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vasiliy.vorobev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vasiliy.vorobev\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,anger,disgust,fear,joy,sadness,surprise,neutral,processed_text
0,"Моя любимая еда — это все, что мне не приходил...",0,0,0,0,0,0,1,любимая еда это приходилось готовить самому
1,"Теперь, если он покончит с собой, все будут ду...",0,0,0,0,0,0,1,покончит собой будут думать смеется трахая люд...
2,КАКОГО НАХРАНА БЭЙЛЕССКАЯ ИЗОИНЦИЯ?,1,0,0,0,0,0,0,какого нахрана бэйлесская изоинция
3,Чтобы она почувствовала угрозу,0,0,1,0,0,0,0,почувствовала угрозу
4,Грязные южные дрочники,1,0,0,0,0,0,0,грязные южные дрочники
...,...,...,...,...,...,...,...,...,...
43405,"Добавил, приятель, у меня только что появился ...",0,0,0,1,0,0,0,добавил приятель появился лук нравится охотнич...
43406,"Всегда думал, что это смешно, но является ли э...",0,0,0,0,0,1,0,думал это смешно является это отсылкой чему либо
43407,"О чем ты говоришь? Во всем плохом, что произош...",1,0,0,0,0,0,0,говоришь всем плохом произошло виноват имя имя...
43408,"Больше похоже на крещение, с сексуальными резу...",0,0,0,1,0,0,0,похоже крещение сексуальными результатами


In [89]:
# 4. Векторизовать поле text TF-IDF
import joblib

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['processed_text'])

# Сохранение векторизатора
joblib.dump(vectorizer, 'C:/Users/vasiliy.vorobev/Documents/Разное/Обучение/Проект/Лог регрессия/vectorizer.joblib')


['C:/Users/vasiliy.vorobev/Documents/Разное/Обучение/Проект/Лог регрессия/vectorizer.joblib']

In [97]:
from sklearn.metrics import f1_score

# Обучение модели


for emotion in emotions:
    y = df[emotion]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LogisticRegression(penalty="l2", fit_intercept=True, max_iter=100, C=1, solver="lbfgs", random_state=42)
    model.fit(X_train, y_train)

    # сохранение модели
    joblib.dump(model, 'C:/Users/vasiliy.vorobev/Documents/Разное/Обучение/Проект/Лог регрессия/model' + emotion + '.joblib')
    
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    report = classification_report(y_val, y_pred)

    print(emotion)
    print(f"accuracy: {accuracy}")
    print(f"f1 score: {f1}")
    print("Отчет по классификации:")
    print(report)

anger
accuracy: 0.8760654227136605
f1 score: 0.1308562197092084
Отчет по классификации:
              precision    recall  f1-score   support

           0       0.88      1.00      0.93      7549
           1       0.77      0.07      0.13      1133

    accuracy                           0.88      8682
   macro avg       0.82      0.53      0.53      8682
weighted avg       0.86      0.88      0.83      8682

disgust
accuracy: 0.9841050449205252
f1 score: 0.11538461538461539
Отчет по классификации:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8536
           1       0.90      0.06      0.12       146

    accuracy                           0.98      8682
   macro avg       0.94      0.53      0.55      8682
weighted avg       0.98      0.98      0.98      8682

fear
accuracy: 0.9847961299239807
f1 score: 0.0958904109589041
Отчет по классификации:
              precision    recall  f1-score   support

           0       0.99  